In [23]:
network_url = 'https://ropsten.infura.io/v3/55a512fc01d84f59aa61929ffc8dff87'

In [ ]:
0xf383c4bf0a5c32dd3369b02f68fd4e4400ef59343ad472bc96a28827f32c9abb

In [24]:
import web3
import json
w3 = web3.Web3(web3.HTTPProvider(network_url))
sender = "0x18648B486Bd6B771DB957590E988A2464F22BfCd"
balance = w3.fromWei(w3.eth.get_balance(sender),"ether")
tx_count = w3.eth.getTransactionCount(sender)
contract_address = "0x0243c7aa552730E8C6F7ED25A480a7C0c88a70f0"
print("Transaction Count {}".format(tx_count))
print("ETH Balance: {}".format(balance))
print("Sender Account: {}".format(sender))
compiled_contract_path = '../sol/build/contracts/FixityStorage.json'
# check contract address if this cell fails
deployed_contract_address = w3.toChecksumAddress(contract_address)
print("Contract Deployed at: {}".format(deployed_contract_address))

with open(compiled_contract_path) as file:
    contract_json = json.load(file)  # load contract info as JSON
    contract_abi = contract_json['abi']  # fetch contract's abi - necessary to call its functions

# Fetch deployed contract reference
contract = w3.eth.contract(address=deployed_contract_address, abi=contract_abi)
print("Contract Functions: {}".format(contract.all_functions()))

Transaction Count 3
ETH Balance: 0.096212689
Sender Account: 0x18648B486Bd6B771DB957590E988A2464F22BfCd
Contract Deployed at: 0x0243c7aa552730E8C6F7ED25A480a7C0c88a70f0
Contract Functions: [<Function getPoolHash(uint32)>, <Function setPoolHash(uint32,bytes32)>]


In [25]:
from Pool import Pool
from Object import Object
from Archive import Archive
import random
import hashlib

encoding = "utf-8"
N = 10000
percent = 10
prevalence = (percent * N) /100.0
p = percent/100.0
k=3

objects = [Object(i,random.uniform(0,p)) for i in range(0,N)]
assert objects[2].hash == hashlib.sha256("2".encode(encoding)).hexdigest()

pools=[]
for i in range((len(objects) + k - 1) // k ):
    # assign pool id to each object in the pool
    for obj in objects[i * k:(i + 1) * k]:
        obj.pool_id=i
    pools.append(Pool(objects[i * k:(i + 1) * k],i))
assert pools[0].id == 0
assert len(pools[0].objects) == len(pools[1].objects)
assert pools[len(pools)-1].hash

In [44]:
poolHashBytes = contract.functions.getPoolHash(0).call()
poolHashBytes

b'\x17`\xd2p\x83\xf6\xe2\xd1\xc4je\x93\x8c\x03\xa0\xc5-\xcc\xf5\\\xb4\xebh\xa7 \xe6\xef\xe3\xa8\x85\x1fx'

In [28]:
# TODO get real world data 
gasPrice = w3.toWei('9.35', 'gwei')

metaTx = {
    "nonce":w3.eth.getTransactionCount(sender),
    "from":sender,
    "to":deployed_contract_address,
    "gasPrice": w3.eth.gas_price
}
print(metaTx)
print(w3.eth.getTransactionCount(sender))



{'from': '0x18648B486Bd6B771DB957590E988A2464F22BfCd', 'to': '0x0243c7aa552730E8C6F7ED25A480a7C0c88a70f0', 'gasPrice': 9350000000}
3


In [31]:
9350000000 
9838614869

9838614869

In [45]:
pool = pools[0]
metaTx.update({"nonce":w3.eth.getTransactionCount(sender)})
tx = contract.functions.setPoolHash(pool.id,pool.hash).buildTransaction(
    {
        "nonce":w3.eth.getTransactionCount(sender),
        "gas": 50000,
        "gasPrice": 9350000000
    }
)

private_key="57b21d9e3d36c19c548091acd195c91ebe3ed27dec0fe4903d008f9af9e72f39"
signed_tx = w3.eth.account.sign_transaction(tx,private_key=private_key)
signed_tx.hash
signed_tx


SignedTransaction(rawTransaction=HexBytes('0xf8a90585022d4dad8082c350940243c7aa552730e8c6f7ed25a480a7c0c88a70f080b844178d292900000000000000000000000000000000000000000000000000000000000000001760d27083f6e2d1c46a65938c03a0c52dccf55cb4eb68a720e6efe3a8851f782aa0e0119937d868c24eb48a4d4244e70447d63c950d60713a10ef7bc158a6af5fc5a07db0a93013399287be15e5aefbbc6da549bbcec6a99bdcd1d764ca3958aabc52'), hash=HexBytes('0xcf70618cba850f25c8a795f3ff3e7e1ff8d84674ced549871a2d1b09219337cb'), r=101349171955531579537692253290274659607135148774002003810273744102708420239301, s=56851238847543959263851460604244997406670721051345708215295078251189477489746, v=42)

In [54]:
w3.eth.get_transaction("0x8839e03f0143bad34060fa909c35d30f2edb22dd4fdac0264de8ae84176eb1ea")

AttributeDict({'blockHash': HexBytes('0xb100ddb2cbe51d1e77bbb779cb46c5c2a334b82d7dec31849fb8929eaf213b0f'),
 'blockNumber': 12145737,
 'from': '0x18648B486Bd6B771DB957590E988A2464F22BfCd',
 'gas': 50000,
 'gasPrice': 9350000000,
 'hash': HexBytes('0x8839e03f0143bad34060fa909c35d30f2edb22dd4fdac0264de8ae84176eb1ea'),
 'input': '0x178d292900000000000000000000000000000000000000000000000000000000000000001760d27083f6e2d1c46a65938c03a0c52dccf55cb4eb68a720e6efe3a8851f78',
 'nonce': 4,
 'r': HexBytes('0x71b9fa00f8680ad937bde979fb4e9eb175d6bdfc181acfbf18fe74ea49d4261e'),
 's': HexBytes('0x42aba6acbc7a836dd33bda25a6428de486f7d339f8a62acc1a637851b353132c'),
 'to': '0x0243c7aa552730E8C6F7ED25A480a7C0c88a70f0',
 'transactionIndex': 16,
 'type': '0x0',
 'v': 41,
 'value': 0})

In [41]:
w3.eth.send_raw_transaction(signed_tx.rawTransaction)

HexBytes('0x8839e03f0143bad34060fa909c35d30f2edb22dd4fdac0264de8ae84176eb1ea')

Compiling your contracts...
===========================
> Compiling .\contracts\FixityStorage.sol
   > transaction hash:    0xf383c4bf0a5c32dd3369b02f68fd4e4400ef59343ad472bc96a28827f32c9abb
   > Blocks: 6            Seconds: 193
   > contract address:    0x0243c7aa552730E8C6F7ED25A480a7C0c88a70f0
   > block number:        12145612
   > block timestamp:     1648547025
   > account:             0x18648B486Bd6B771DB957590E988A2464F22BfCd
   > balance:             0.096212689
   > gas used:            166079 (0x288bf)
   > gas price:           9.35 gwei
   > value sent:          0 ETH
   > total cost:          0.00155283865 ETH

   Pausing for 2 confirmations...

   -------------------------------
   > confirmation number: 1 (block: 12145614)
   > confirmation number: 2 (block: 12145615)
   > Saving artifacts
   -------------------------------------
   > Total cost:       0.00155283865 ETH

Summary
=======
> Total deployments:   1
> Final cost:          0.00155283865 ETH

In [10]:
from Pool import Pool
from Object import Object
from Archive import Archive
import random
import hashlib

encoding = "utf-8"
N = 100
percent = 10
prevalence = (percent * N) /100.0
p = percent/100.0
k=3

In [11]:
objects = [Object(i,random.uniform(0,p)) for i in range(0,N)]
assert objects[2].hash == hashlib.sha256("2".encode(encoding)).hexdigest()

pools=[]
for i in range((len(objects) + k - 1) // k ):
    # assign pool id to each object in the pool
    for obj in objects[i * k:(i + 1) * k]:
        obj.pool_id=i
    pools.append(Pool(objects[i * k:(i + 1) * k],i))
assert pools[0].id == 0
assert len(pools[0].objects) == len(pools[1].objects)
assert pools[len(pools)-1].hash

In [14]:
pools[0].hash

'1760d27083f6e2d1c46a65938c03a0c52dccf55cb4eb68a720e6efe3a8851f78'